In [15]:
with open('requirements.txt', 'w') as f:
    f.write('''scipy
pandas
matplotlib
tensorflow
Keras-Preprocessing
torchvision
scikit-learn>=1.3
''')

In [16]:
!pip install -r requirements.txt

     -------------------------------------- 42.6/42.6 kB 344.1 kB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

In [1]:
import torch
from torchvision import models
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
# Load resnet18 model
resnet18_model = models.resnet18(pretrained=False)  # Set pretrained to False
num_features = resnet18_model.fc.in_features
resnet18_model.fc = nn.Linear(num_features, 102)  # Adjust the final layer for 102 classes

# Move model to device
resnet18_model = resnet18_model.to(device)

# Step 2: Load the saved model weights
resnet18_model.load_state_dict(torch.load('resnet_18(5).pth'))

# Set the model to evaluation mode
resnet18_model.eval()

print("ResNet18 model loaded successfully from 'model_resnet18.pth'")

ResNet18 model loaded successfully from 'model_resnet18.pth'


In [4]:
# Load vgg16 model

vgg16_model = models.vgg16(pretrained=False)  # Set pretrained to False
num_features = vgg16_model.classifier[6].in_features
vgg16_model.classifier[6] = nn.Linear(num_features, 102)  # Adjust the final layer for 102 classes

# Move model to device
vgg16_model = vgg16_model.to(device)

# Step 2: Load the saved model weights
vgg16_model.load_state_dict(torch.load('model_vgg16(8).pth'))

# Set the model to evaluation mode
vgg16_model.eval()

print("VGG16 model loaded successfully from 'model_vgg16.pth'")

VGG16 model loaded successfully from 'model_vgg16.pth'


In [14]:
# Load vgg19 model

vgg19_model = models.vgg19(pretrained=False)  # Set pretrained to False
num_features = vgg19_model.classifier[6].in_features
vgg19_model.classifier[6] = nn.Linear(num_features, 102)  # Adjust the final layer for 102 classes

# Move model to device
vgg19_model = vgg19_model.to(device)

# Step 2: Load the saved model weights
vgg19_model.load_state_dict(torch.load('model_vgg19(13).pth'))

# Set the model to evaluation mode
vgg19_model.eval()

print("VGG19 model loaded successfully from 'model_vgg19.pth'")

VGG19 model loaded successfully from 'model_vgg19.pth'


In [6]:
# Load DeiT model
# Define the path to your saved model weights
from transformers import DeiTForImageClassification
model_deit_path = 'model_deit(2).pth'

# Load DEIT model architecture from Torch Hub
deit_model = DeiTForImageClassification.from_pretrained(
        'facebook/deit-base-distilled-patch16-224',
        num_labels=102
    )

# Load the saved model weights
deit_model.load_state_dict(torch.load(model_deit_path))

# Set the model to evaluation mode
deit_model.eval()

print("DEIT model loaded successfully from 'model_deit.pth'")


Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DEIT model loaded successfully from 'model_deit.pth'


In [7]:
# Test Data

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transformations for the test dataset
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet18 expects 224x224 input size
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Standard normalization for ResNet
])

# Load the test dataset
test_folder = '102flowers_segmen_split/test'
test_dataset = ImageFolder(test_folder, transform=test_transforms)

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [15]:
# Move deit_model to the appropriate device (GPU if available)
deit_model = deit_model.to(device)

# Initialize lists to store probabilities and true labels across batches
prob_resnet_all = []
prob_deit_all = []
prob_all_ensemble = []
true_labels = []

# Evaluate with no gradient calculations
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Get probabilities from each model and ensure outputs are on the same device
        prob_vgg16 = nn.functional.softmax(vgg16_model(inputs.to(device)), dim=1)
        prob_vgg19 = nn.functional.softmax(vgg19_model(inputs.to(device)), dim=1)
        prob_resnet_4 = nn.functional.softmax(resnet18_model(inputs.to(device)), dim=1)
        prob_deit = nn.functional.softmax(deit_model(inputs.to(device)).logits, dim=1)  # Extract logits for DeiT

        # Calculate the average probabilities for each ensemble method
        avg_prob_resnet = (prob_vgg16 + prob_vgg19 + prob_resnet_4) / 3
        avg_prob_deit = (prob_vgg16 + prob_vgg19 + prob_deit) / 3
        avg_prob_all = (prob_vgg16 + prob_vgg19 + prob_resnet_4 + prob_deit) / 4

        # Append to respective lists
        prob_resnet_all.append(avg_prob_resnet)
        prob_deit_all.append(avg_prob_deit)
        prob_all_ensemble.append(avg_prob_all)
        true_labels.append(labels)

# Concatenate all stored tensors
prob_resnet_all = torch.cat(prob_resnet_all)
prob_deit_all = torch.cat(prob_deit_all)
prob_all_ensemble = torch.cat(prob_all_ensemble)
true_labels = torch.cat(true_labels)

# Calculate final predictions
final_preds_resnet = torch.argmax(prob_resnet_all, dim=1)
final_preds_deit = torch.argmax(prob_deit_all, dim=1)
final_preds_all = torch.argmax(prob_all_ensemble, dim=1)


In [16]:
# Calculate accuracy
correct_resnet = torch.sum(final_preds_resnet == true_labels).item()
correct_deit = torch.sum(final_preds_deit == true_labels).item()
correct_all = torch.sum(final_preds_all == true_labels).item()

total = len(true_labels)
accuracy_resnet = correct_resnet / total * 100
accuracy_deit = correct_deit / total * 100
accuracy_all = correct_all / total * 100

print(f"ResNet18 Ensemble Accuracy: {accuracy_resnet:.4f}")
print(f"DeiT Ensemble Accuracy: {accuracy_deit:.4f}")
print(f"Combined Ensemble Accuracy: {accuracy_all:.4f}")


ResNet18 Ensemble Accuracy: 76.6629
DeiT Ensemble Accuracy: 80.0130
Combined Ensemble Accuracy: 80.7123
